Visualization is based on a plotly example that can be found here: https://plot.ly/python/3d-network-graph/


In [1]:
# maybe put all this in __init__.py ?
import gently as mz
import gently.physics 
import gently.simulation 
import gently.geometry 
import gently.visualization 
import gently.metrics 
import gently.terrain

import networkx as nx
#Plotly seems very nice but I think it's not very open dand kinda commercial so keep an eye on that!
import plotly as py
import plotly.graph_objs as go
import math
import copy
import random
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np
#import scipy.stats as ss
import pyswarms as ps
import itertools

In [2]:
# Global Parameters (here so can be played around easily)
# Random seed is fixed for reproducibility
random.seed(42) # This doesn't fix for scipy.stats so moved everything to random. 
# scipy.stats wasn't really helping with anything anyway, removed one dependency this way...


# Population Paratemeters
N = 400
N_building_frac = 150/400; # This is the number of nodes that are reserved solely for buildings.
                                # Buildings may (likely) end up with more nodes due to reg. nodes falling in buildings
N_building_reserved = int(N*N_building_frac)
N_node = N-N_building_reserved # excluding building ones, they are generated seperately.
N_b = 9 # let's try 6 APs, so optimization is a bit easier...

# Sim Parameters
d_basestation = 0.125 # AP comm range, normalized
d_node = 0.001   # node comm range, normalized
pcost_node = 1  # Cost of node-node transmission
pcost_basestation = 1.5  # cost of node-AP transmission
dim = 5; #3D

# Optimization Parameters
n_swarm = 200
n_iter = 75

# about saving and stuff
file_name = '2D_Nb' + str(N_b) + '_db' + str(d_basestation) + '_dn' + str(d_node) + '_' + str(n_swarm) + '_' + str(n_iter)


In [3]:
## Generate buildings first so buildings can be filled accordingly.

# @TODO: Automatize this by having a grid and deciding if to place a building at each grid point via a probability 
# and randomizing the sizes. Fully random can be done by doing 2D rectangle-rectange collusion checks, however
# grid is more realistic for US Urban scenario than a random mess. Thats more of a continental thing :)

base_origin = mz.geometry.Point(0.2, 0.3, 0)
base_size = {'length':0.1, 'width':0.15}
height = 0.2

#                    {'base':{'x': 0.45, 'y':0.6, 'z':0},'size':{'length':0.2, 'width':0.1},'height':0.05},
pert_param_list = [ {'base':mz.geometry.Point(0.2,0.55,0)  ,'size':{'length':0.1, 'width':0.15}    ,'height': 0.085},
                    {'base':mz.geometry.Point(0.6,0.35,0)  ,'size':{'length':0.1, 'width':0.1}     ,'height':0.125},
                    {'base':mz.geometry.Point(0.3,0.2,0)   ,'size':{'length':0.15, 'width':0.2}    ,'height':0.065},
                    {'base':mz.geometry.Point(0.7,0.8,0)   ,'size':{'length':0.075, 'width':0.1}   ,'height':0.1},
                    {'base':mz.geometry.Point(0.15,0.35,0) ,'size':{'length':0.075, 'width':0.1}   ,'height':0.06},
                    {'base':mz.geometry.Point(0.45,0.65,0) ,'size':{'length':0.125, 'width':0.1}   ,'height':0.060},
                    {'base':mz.geometry.Point(0.45,0.45,0) ,'size':{'length':0.085, 'width':0.095} ,'height':0.075}
                  ]

pert_list = [mz.geometry.RectPrism(p['base'],p['size'],p['height']) for p in pert_param_list]

In [4]:
## First generate the population

# Populate Nodes
# Generate the x-y coordinates (@TODO we also need to make sure there are no duplicate 3D coordinates)
mu = 0;
rho = 5;
#x = ss.norm.rvs(loc=mu,scale=rho,size=N)

#x = np.array([random.gauss(mu,rho) for n in np.arange(N_node)])
#x = [s + abs(min(x)) for s in x]
# Trying with uniform dist.
x = np.array([random.uniform(0, 1) for n in np.arange(N_node)])


#y = ss.norm.rvs(loc=mu,scale=rho,size=N)
#y = np.array([random.gauss(mu,rho) for n in np.arange(N_node)])
#y = [s + abs(min(y)) for s in y]
# Trying with uniform dist.
y = np.array([random.uniform(0, 1) for n in np.arange(N_node)])


# generate nodes that will be inside the buildings. Can put them on base points of buildings, they will get 
# mixed into the building when conform function is called.
x_building = np.array([])
y_building = np.array([])
h_building = np.array([])
    
#sf = max(max(x),max(y))
#x = x/sf;
#y = y/sf;

# to make it work with uniform
sf = 25

# These are already normalized, so don't do it twice.
for idx in np.arange(N_building_reserved):
    pert = random.choice(pert_list)
    x_building = np.append(x_building,pert.base_origin.x)
    y_building = np.append(y_building,pert.base_origin.y)
    h_building = np.append(h_building,pert.base_origin.z)
# append new building nodes into the lists
x = np.append(x,x_building)
y = np.append(y,y_building)


# Generate z axis(height), heavily around zero.
lambd = 10
low = 0
high = 0.1

#h = ss.expon.rvs(loc=0,scale=1/lambd,size=N)
h = np.array([random.expovariate(lambd) for n in np.arange(N_node)])
h = h/(sf*2)*2 # @TODO: Hack Solution! Fix sampling of height!!
# append new building nodes into the lists, here because they are already normalized
h = np.append(h,h_building)
# Normalize the coordinates to be in a unit cube
node_coordinates = (x,y,h)

# Populate basestations

bs_range = (0.1,0.9) # Outlying box of AP rectangle
h_bs = 0.005; # starting height for APs

x_list = np.linspace(bs_range[0],bs_range[1],int(math.sqrt(N_b)))
y_list = np.linspace(bs_range[0],bs_range[1],int(math.sqrt(N_b)))
xx, yy = np.meshgrid(x_list, y_list)

x_basestation = [] 
y_basestation = []
for i in range(len(xx)):
    for j in range(len(yy)):
        x_basestation.append(xx[i,j]) 
        y_basestation.append(yy[i,j])

x_basestation = np.array(x_basestation)
y_basestation = np.array(y_basestation)

#sf = max(max(x_basestation),max(y_basestation))
#x_basestation = x_basestation/sf
#y_basestation = y_basestation/sf

#h_basestation = np.ones(N_b) * np.mean(node_coordinates[2])
h_basestation = np.ones(N_b)*h_bs

basestation_coordinates = (x_basestation,y_basestation,h_basestation) # couldn't do this in a better way for some reason.

# Convert into dictionary and give every vertex their unique id
node_keys = np.arange(N)
node_coordinates_dict = {i: (node_coordinates[0][i],
                             node_coordinates[1][i],
                             node_coordinates[2][i]) for i in node_keys}
# Dictionary keys will also be node id's so have to make them unique
basestation_keys = np.arange(N,N+N_b)
basestation_coordinates_dict = {i: (basestation_coordinates[0][i-N],
                                    basestation_coordinates[1][i-N],
                                    basestation_coordinates[2][i-N]) for i in basestation_keys}

In [5]:
## Construct the network and prepare the stage

coordinates = {'node': node_coordinates_dict, 'basestation': basestation_coordinates_dict}
radii = {'node': d_node, 'basestation': d_basestation}
costs = {'node': pcost_node, 'basestation': pcost_basestation}

S = mz.simulation.SimStage(coordinates,radii,costs)

G = S.G_dict['combined']

In [6]:
## Decide on the terrain and update the stage accordingly.

t_params = {'range': {'low':0.3,'high':0.75},'height':0.1}
S.update_terrain(mz.terrain.surface_height_func4,t_params)

In [7]:
%%time
## Update the stage with buildings 
S.update_perturbations(pert_list) #0ns
S.conform_node_heights(['node','basestation']) #41ms
S.update_connections()

Wall time: 532 ms


In [8]:
## Visualize to see what everything looks like
# Enter the params for the network to be visualized
mz.visualization.visualize_stage(S,figure_name='module_test_' +file_name)

In [9]:
## Compute Initial performance metrics
G_met = S.G_dict['combined']

is_conn = mz.metrics.is_connected(G_met)
if is_conn == True:
    print('network IS connected')
else: 
    print('network is NOT connected')

bs_keys = S.coordinates_dict_base['basestation'].keys()
conn_frac = mz.metrics.connected_fraction(G_met,True,bs_keys)
print('percentage connected:' + str(conn_frac))

conn_k = mz.metrics.k_connectedness(G_met)
print('k-connectedness k:' + str(conn_k[1]))

avg_path = mz.metrics.avg_shortest_path_length(G_met)
print('average shortest path length:' + str(avg_path[1]))

avg_path_weighted = mz.metrics.avg_shortest_path_length(G_met,'power_cost')
print('average shortest path length(weighted):' + str(avg_path_weighted[1]))


network is NOT connected
percentage connected:0.2675
k-connectedness k:1
average shortest path length:1.9261083743842364
average shortest path length(weighted):2.8866995073891624


In [12]:
# Grid search is not doable, going with Particle Swarm
def network_sim_2d(params):
    """Network Simulation as objective function

    This computes a network simulation for given AP locations
    and returns the goal, which is the connected percentage

    Inputs
    ------
    params: np.ndarray
        Unrolled version of the coordinates for each access point
        ((x1,y1),(x2,y2),...,h) last one is the height for all, since
        this is 2D

    Returns
    -------
    float
        The computed unconnected percentage of the network 
        (unconnected because expects cost to return)
    """
    # Unroll the received parameters into coordinates
    
    h = params[-1] # height is the last element
    #print(params)
    #print(h)
    n_axes = 2 # only (x,y)
    n_aps = int((len(params)-1)/n_axes)
    # excluding last element from reshape
    xy_array = params[:-1].reshape((n_aps,n_axes)) 
    #print(xy_array)
    x_array = xy_array[:,0]
    y_array = xy_array[:,1]
    #print(x_array)
    #print(y_array)
    h_array = np.empty(n_aps);
    h_array.fill(h) # create a list of same height (2D approx.) for all APs
    
    # Perform the network simulation
    
    # Here should check if all basestation coordinates are viable, and continue to next coordinate if not.
    # this will seriously mess with the optimizer though, I don't know what to do about that. Maybe use discrete
    # values.

    S.update_node_coordinates(x_array,y_array,h_array,n_type='basestation')
    S.update_connections()
    G_met = S.G_dict['combined']
    # avg length of shortest paths, LINK WEIGHTED 
    # First check if the network is connected 
    
    # find the basestations that are ABOVE ground.
    viable_bs_list = list()
    for bs_key in S.coordinates_dict['basestation'].keys():
        c_p = S.coordinates_dict['basestation'][bs_key] #height
        c_h = c_p[2]
        t_h = S.terrain_function(mz.geometry.Point(c_p[0],c_p[1],0), S.terrain_params)
        
        if t_h <= c_h:
            viable_bs_list.append(bs_key)
            
    # Compute the cost
    conn_frac = mz.metrics.connected_fraction(G_met,True,viable_bs_list) 
    # conn_frac is gain, need to return loss
    loss = 1 - conn_frac # fraction of unconnected nodes
    #print(loss)
    return loss

def f_2d(x):
    """Higher-level method to do the simulation for the
    whole swarm.

    Inputs
    ------
    x: numpy.ndarray of shape (n_particles, dimensions)
        The swarm that will perform the search

    Returns
    -------
    numpy.ndarray of shape (n_particles, )
        The computed loss for each particle
    """
    #print('selams')
    n_particles = x.shape[0] # this can be parallelized...
    j = [network_sim_2d(x[i]) for i in range(n_particles)]
    #print(x[0])
    #print(x[1])
    #print(str(j[0]) + ' ' + str(j[1]) + ' ' + str(j[2]) + ' ' + str(j[3]))
    return np.array(j)

In [13]:
%%time
# bounds, not my best but I'm really sleepy...
max_bound = 1 * np.ones(N_b*2)#xy bounds
min_bound = np.zeros(N_b*2)   #xy bounds
max_bound = np.append(max_bound,0.125) #h bound
min_bound = np.append(min_bound,0.025)  #h bound

bounds = (min_bound, max_bound)


# Initialize swarm, arbitrary for now. This can also be searched for apparently...
#options = {'c1': 0.5, 'c2': 0.3, 'w':0.3} #optimizing this didn't work very well for some reason, check that again.
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.4, 'k': 20, 'p': 2}
# Call instance of PSO, also remember constraints
dimensions = ((2 * N_b) + 1)
optimizer = ps.single.LocalBestPSO(n_particles=n_swarm, dimensions=dimensions, options=options, bounds=bounds)

# Perform optimization # each iteration appears to be the same, warum?
cost, pos = optimizer.optimize(f_2d, print_step=1, iters=n_iter, verbose=3)

INFO:pyswarms.single.local_best:Iteration 1/75, cost: 0.6699999999999999
INFO:pyswarms.single.local_best:Iteration 2/75, cost: 0.6699999999999999
INFO:pyswarms.single.local_best:Iteration 3/75, cost: 0.6699999999999999
INFO:pyswarms.single.local_best:Iteration 4/75, cost: 0.6475
INFO:pyswarms.single.local_best:Iteration 5/75, cost: 0.64
INFO:pyswarms.single.local_best:Iteration 6/75, cost: 0.64
INFO:pyswarms.single.local_best:Iteration 7/75, cost: 0.64
INFO:pyswarms.single.local_best:Iteration 8/75, cost: 0.635
INFO:pyswarms.single.local_best:Iteration 9/75, cost: 0.63
INFO:pyswarms.single.local_best:Iteration 10/75, cost: 0.6125
INFO:pyswarms.single.local_best:Iteration 11/75, cost: 0.6125
INFO:pyswarms.single.local_best:Iteration 12/75, cost: 0.585
INFO:pyswarms.single.local_best:Iteration 13/75, cost: 0.5800000000000001
INFO:pyswarms.single.local_best:Iteration 14/75, cost: 0.5725
INFO:pyswarms.single.local_best:Iteration 15/75, cost: 0.5625
INFO:pyswarms.single.local_best:Iteration

Wall time: 4h 20min 2s


In [14]:
import pickle
# Obtain the cost history
cost_history_2d = optimizer.get_cost_history
# Obtain the position history
pos_history_2d = optimizer.get_pos_history
# Obtain the velocity history
velocity_history = optimizer.get_velocity_history

pickle.dump((cost_history_2d,pos_history_2d,velocity_history),open('opt_results_'+ file_name +'.p','wb'))

In [15]:
h = pos[-1] # height is the last element
#print(params)
#print(h)
n_axes = 2 # only (x,y)
n_aps = int((len(pos)-1)/n_axes)
# excluding last element from reshape
xy_array = pos[:-1].reshape((n_aps,n_axes)) 
#print(xy_array)
x_array = xy_array[:,0]
y_array = xy_array[:,1]
#print(x_array)
#print(y_array)
h_array = np.empty(n_aps);
h_array.fill(h) # create a list of same height (2D approx.) for all APs

S.update_node_coordinates(x_array,y_array,h_array,n_type='basestation')
S.update_connections()
G_met = S.G_dict['combined']
# avg length of shortest paths, LINK WEIGHTED 
# First check if the network is connected 

# Compute the cost
conn_frac = mz.metrics.connected_fraction(G_met,True,S.coordinates_dict_base['basestation'].keys()) 

mz.visualization.visualize_stage(S,figure_name='opt_result_' +file_name)